# **Modeling and Evaluation**

## Objectives

* Fit and evaluate a model pipeline to predict chargers

## Inputs

* Engineered features 'output/datasets/engineered/insurance_engineered.csv' 

## Outputs

* Model pipeline


---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load Engineered Dataset

In [ ]:
import pandas as pd
df_path = 'outputs/datasets/engineered/insurance_engineered.csv'
df_processed = pd.read_csv(df_path)
df_processed.head()

---

Disable warnings

# Train/Test Split

Split features and target

In [ ]:
# Features (X) and Target (y)
X = df_processed.drop('charges', axis=1)
y = df_processed['charges']

Train/Test split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=2)

print(f"X_train shape: {X_train.shape}  y_train shape: {y_train.shape}"
      f"\nX_test shape: {X_test.shape}  y_test shape: {y_test.shape}")

---

# Define Hyperparameters and Models

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

models = {
    "Linear Regression": {
        "model": LinearRegression(),
        "params": {
            "fit_intercept": [True, False],
            "normalize": [False]
        }
    },
    "Ridge Regression": {
        "model": Ridge(),
        "params": {
            "alpha": [0.01, 0.1, 1, 10, 100],
            "fit_intercept": [True, False]
        }
    },
    "Random Forest": {
        "model": RandomForestRegressor(random_state=2),
        "params": {
            "n_estimators": [100, 200],
            "max_depth": [None, 10, 20],
            "min_samples_split": [2, 5],
        }
    },
    "XGBoost": {
        "model": XGBRegressor(objective='reg:squarederror', random_state=2),
        "params": {
            "n_estimators": [100, 200],
            "max_depth": [3, 6],
            "learning_rate": [0.01, 0.1],
            "subsample": [0.8, 1.0]
        }
    }
}


## Train and Evaluate All Models

In [ ]:
best_model_name = None
best_model_obj = None
best_score = -np.inf 

results = []

for name, cfg in models.items():
    print(f"Training {name}...")
    if name == "Linear Regression" and "normalize" in cfg["params"]:
        del cfg["params"]["normalize"]
    
    grid = GridSearchCV(cfg["model"], cfg["params"], cv=5, scoring="r2", n_jobs=-1)
    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)

    if r2 > best_score:
        best_score = r2
        best_model_name = name
        best_model_obj = best_model

    results.append({
        "Model": name,
        "R²": round(r2, 3),
        "RMSE": round(rmse, 2),
        "MAE": round(mae, 2)
    })



Show Results Table

In [ ]:
import matplotlib.pyplot as plt

def plot_tree_model_features(model, feature_names, title):
    importances = model.feature_importances_
    sorted_idx = np.argsort(importances)[::-1]
    top_features = [feature_names[i] for i in sorted_idx]
    top_importances = importances[sorted_idx]

    plt.figure(figsize=(10, 6))
    plt.barh(top_features, top_importances)
    plt.gca().invert_yaxis()
    plt.title(title)
    plt.xlabel("Feature Importance")
    plt.grid(True)
    plt.show()

def plot_linear_model_coefficients(model, feature_names, title):
    coefs = model.coef_
    sorted_idx = np.argsort(np.abs(coefs))[::-1]
    top_features = [feature_names[i] for i in sorted_idx]
    top_coefs = coefs[sorted_idx]

    plt.figure(figsize=(10, 6))
    plt.barh(top_features, top_coefs)
    plt.gca().invert_yaxis()
    plt.title(title)
    plt.xlabel("Coefficient Value")
    plt.grid(True)
    plt.show()

def get_feature_importance_df(model, feature_names, model_type):
    if model_type in ["Random Forest", "XGBoost"]:
        values = model.feature_importances_
    elif model_type in ["Linear Regression", "Ridge Regression"]:
        values = model.coef_
    else:
        raise ValueError("Unsupported model type for feature importance extraction.")

    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': values
    })

    importance_df['Abs_Importance'] = importance_df['Importance'].abs()
    importance_df.sort_values(by='Abs_Importance', ascending=False, inplace=True)
    importance_df.drop(columns='Abs_Importance', inplace=True)
    importance_df.reset_index(drop=True, inplace=True)

    return importance_df


In [ ]:
results_df = pd.DataFrame(results)
results_df.sort_values(by="R²", ascending=False, inplace=True)
results_df.reset_index(drop=True, inplace=True)

print("Model Evaluation Results:")
print(results_df)

* GridSearchCV uses 5-fold cross-validation by default
* We tested 4 models:
  * Linear Regression
  * Random Forest Regressor
  * Gradient Boosting Regressor
  * XGBoost Regressor
* All the models perfomed good, but the best model is XGBoost Regressor with a R2 score of 0.87

## Feature Importance

In [ ]:
feature_names = X_train.columns.tolist()

# Get and display the importance table
importance_df = get_feature_importance_df(best_model_obj, feature_names, best_model_name)
print(f"\nFeature Importance Ranking for {best_model_name}:\n")
print(importance_df.to_string(index=True))

plot_tree_model_features(best_model_obj, feature_names, f"{best_model_name} - Feature Importance")



## Refit pipeline with the Best Features

The feature importance table and plot shows the most important features for the model. We can see that the `risk_score, smoker_bmi_risk, smoker_encoded, age, bmi, children, age_bmi_risk` are the most important features for the model. However, we can see that the `age_squared, is_overweight, age_group_*, bmi_class_*, most region_*, sex_encoded` are not important features for the model. This means that we can remove these features from the dataset and retrain the model to see if we can improve the performance.

In [ ]:
features_to_drop = ["age_squared", "is_overweight", 
    "age_group_Senior", "bmi_class_Obese", "age_group_Middle-aged",
    "region_northwest", "region_southeast", 
    "age_group_Adult", "sex_encoded",
    "bmi_class_Normal", "bmi_class_Overweight", "region_southwest"]

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Drop low-importance features
X_reduced = X.drop(columns=[col for col in features_to_drop if col in X.columns])

# Re-split with new feature set
from sklearn.model_selection import train_test_split
X_train_red, X_test_red, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

# Retrain the same model (use best_model_obj which is XGBoost)
best_model_obj.fit(X_train_red, y_train)

# Predict with reduced features
y_pred_red = best_model_obj.predict(X_test_red)

# Evaluate
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
r2_red = r2_score(y_test, y_pred_red)
rmse_red = np.sqrt(mean_squared_error(y_test, y_pred_red))
mae_red = mean_absolute_error(y_test, y_pred_red)

print("Evaluation After Dropping Low-Importance Features:")
print(f"R² Score:  {r2_red:.3f}")
print(f"RMSE:      ${rmse_red:.2f}")
print(f"MAE:       ${mae_red:.2f}")

After retraining the model with the reduced dataset, we can see that the model performance had a slight improvement, with a R2 score of 0.874 (previusly 0.870). This means that the model is able to explain 87.4% of the variance in the target variable, which is a good result.

## Assess Feature Importance

In [ ]:
feature_names = X_reduced.columns.tolist()

# Get and display the importance table
importance_df = get_feature_importance_df(best_model_obj, feature_names, best_model_name)
print(f"\nFeature Importance Ranking for {best_model_name}:\n")
print(importance_df.to_string(index=True))

plot_tree_model_features(best_model_obj, feature_names, f"{best_model_name} - Feature Importance")


---

# Feature Engineering Transformation

In [ ]:
def final_feature_engineering(df):  # your full engineering function

    # Drop low-importance features
    drop_cols = [
        "age_squared", "is_overweight", 
        "age_group_Senior", "bmi_class_Obese", "age_group_Middle-aged",
        "region_northwest", "region_southeast", 
        "age_group_Adult", "sex_encoded",
        "bmi_class_Normal", "bmi_class_Overweight", "region_southwest"
    ]
    df = df.drop(columns=[col for col in drop_cols if col in df.columns])
    
    # Separate X and y
    X = df.drop(columns=["charges"])
    y = df["charges"]

    return X, y

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureEngineeringWrapper(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self  # no fitting needed

    def transform(self, X):
        X_transformed, _ = final_feature_engineering(X)
        return X_transformed

In [ ]:
from sklearn.pipeline import Pipeline

# Initialize pipeline
final_pipeline = Pipeline([
    ('feature_engineering', FeatureEngineeringWrapper()),
    ('model', XGBRegressor(objective='reg:squarederror', random_state=42, 
                           n_estimators=200, max_depth=6, learning_rate=0.1, subsample=1.0))
])

In [ ]:
_, y_all = final_feature_engineering(df_processed)
final_pipeline.fit(df_processed, y_all)

Now we have a full reusable pipeline that can be used to transform the data and train the model. We can use this pipeline to transform new data and make predictions.

---

# Push files to Repo

In [ ]:
import joblib
import os

version = 'v1'
file_path = f"outputs/ml_pipeline/predict_insurance_cost/{version}"

try:
    os.makedirs(name=file_path, exist_ok=True)
except Exception as e:
    print(e)


### Save train/test set

In [ ]:
# Save train/test sets as CSV
X_train_red.to_csv(f"{file_path}/X_train.csv", index=False)
X_test_red.to_csv(f"{file_path}/X_test.csv", index=False)
y_train.to_csv(f"{file_path}/y_train.csv", index=False)
y_test.to_csv(f"{file_path}/y_test.csv", index=False)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import joblib

# Define your feature engineering as a FunctionTransformer
def manual_feature_engineering(X):
    return X.drop(columns=features_to_drop)

feature_pipeline = Pipeline(steps=[
    ('custom_features', FunctionTransformer(manual_feature_engineering, validate=False))
])

# Save pipeline
joblib.dump(feature_pipeline, f"{file_path}/feature_pipeline.pkl")

## Save feature importance plot

In [ ]:
def save_tree_feature_importance_plot(model, feature_names, model_name, file_path):
    importances = model.feature_importances_
    sorted_idx = np.argsort(importances)[::-1]
    top_features = [feature_names[i] for i in sorted_idx]
    top_importances = importances[sorted_idx]

    plt.figure(figsize=(10, 6))
    plt.barh(top_features, top_importances)
    plt.gca().invert_yaxis()
    plt.title(f"{model_name} - Feature Importance")
    plt.xlabel("Importance")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"{file_path}/feature_importance_{model_name}.png")
    plt.close()

In [ ]:
# Save the appropriate plot
save_tree_feature_importance_plot(best_model_obj, X_reduced.columns.tolist(), best_model_name, file_path)


## Save model pipeline

In [ ]:
joblib.dump(final_pipeline, f"{file_path}/clf_pipeline.pkl")

---

# Conclusion

We have successfully built a model pipeline to predict insurance charges using the engineered features. The best model was the XGBoost Regressor, which achieved a R2 score of 0.874. We also assessed the feature importance and retrained the model with the most important features, achieving a slight improvement in performance. The final model pipeline is ready for use in making predictions on new data.